In [ ]:
# gcloud init
# gcloud auth login
# gcloud services enable \
#   compute.googleapis.com \
#   iam.googleapis.com \
#   iamcredentials.googleapis.com \
#   monitoring.googleapis.com \
#   logging.googleapis.com \
#   notebooks.googleapis.com \
#   aiplatform.googleapis.com \
#   bigquery.googleapis.com \
#   artifactregistry.googleapis.com \
#   cloudbuild.googleapis.com \

In [ ]:
import os
# Retrieve and set PROJECT_ID and REGION environment variables.
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
REGION = os.environ.get('REGION')

PROJECT_ID, REGION

In [ ]:
ARTIFACT_REPOSITORY=os.environ.get('REPOSITORY_NAME')
ARTIFACT_REPOSITORY

In [ ]:
GCS_BUCKET = f"{PROJECT_ID}-bucket"
!gsutil mb -l $REGION gs://$GCS_BUCKET

In [ ]:
# Create an Artifact Repository using the gcloud CLI.
!gcloud artifacts repositories create $ARTIFACT_REPOSITORY \
--repository-format=docker \
--location=$REGION \
--description="Artifact registry for fastapi test"

In [ ]:
IMAGE_URI=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{ARTIFACT_REPOSITORY}/{IMAGE_NAME}:{IMAGE_TAG}"